In this notebook we're going to test the hypothesis that pretraining a model on domain data results in better performance of the model when finetuning a model

The three models we'll be using are:
1. FinBERT 2 v_1
2. FinBERT 2 v_2
3. DistilBERT

Note that the FinBERT's are based on the DistilBERT models
The evaluation metric we'll use are Accuracy, cross entropy loss and macro F1 average.

In [1]:
pip install -U datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 160.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4


In [2]:
from datasets import Dataset, load_dataset, DatasetDict
from transformers import AutoModelForMaskedLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling, Pipeline
from huggingface_hub import login
import os

#login(os.getenv("MACBOOK_HF_KEY"))

In [13]:

finetune_ds = load_dataset('takala/financial_phrasebank', 'sentences_50agree', trust_remote_code=True)

tokenized_dataset = {}

#Setting up the basic info for each model
configs = [
    {'name': 'bert_base', 'model': 'distilbert/distilbert-base-uncased', 'tokenizer': AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')},
    {'name': 'finBERT2_v2', 'model': 'Czunzun/finBERT2_v2', 'tokenizer': AutoTokenizer.from_pretrained('Czunzun/finBERT2_v2')}
]


#Tokenizing each dataset
for config in configs:
    tokenized_dataset[config['name']] = finetune_ds.map(lambda train: config['tokenizer'](train['sentence'], padding="max_length", truncation=True))




#Splitting each tokenized dataset for test,train,eval split with standard 80% train
for dataset in tokenized_dataset:
    traintest_split = tokenized_dataset[dataset]['train'].train_test_split(test_size=.2)
    eval_split = traintest_split['test'].train_test_split(test_size=.5)
    ds = DatasetDict({
        'train': traintest_split['train'],
        'test': traintest_split['test'],
        'eval': eval_split['train']
    })
    tokenized_dataset[dataset] = ds


In [6]:
#computing loss metrics
def compute_metrics(pred):
    from sklearn.metrics import accuracy_score, log_loss, f1_score
    import numpy as np

    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    probs = pred.predictions

    if probs.max() > 1 or probs.min() < 0:
        probs = np.exp(probs) / np.sum(np.exp(probs), axis=-1, keepdims=True)  # softmax


    accuracy = accuracy_score(labels, preds)
    cross_entropy = log_loss(labels, probs)
    f1 = f1_score(labels, preds, average='macro')
    return {
        'accuracy': accuracy,
        'cross_entropy': cross_entropy,
        'f1': f1
    }




#Setting up the training arguments
def training_step(config):
    from transformers import DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer


    data_collator = DataCollatorWithPadding(tokenizer=config['tokenizer'])
    model = AutoModelForSequenceClassification.from_pretrained(config['model'], num_labels=3)

    training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    tpu_num_cores=1,
    dataloader_pin_memory=True,
    dataloader_num_workers=4
    )

    trainer=Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        processing_class=config['tokenizer'],
        compute_metrics=compute_metrics,
        train_dataset=tokenized_dataset[config['name']]['train'],
        eval_dataset=tokenized_dataset[config['name']]['test']
        )


    trainer.train()
    results = trainer.evaluate()

    print(results)

In [14]:
#Initialize training for both datasets
for config in configs:
    #Training
    training_step(config=config)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.691800
1000,0.550800
1500,0.367600
2000,0.364200
2500,0.158700


{'eval_loss': 0.8303661346435547, 'eval_accuracy': 0.8402061855670103, 'eval_cross_entropy': 0.823719762512187, 'eval_f1': 0.8083342887148938, 'eval_runtime': 3.8259, 'eval_samples_per_second': 254.057, 'eval_steps_per_second': 63.514, 'epoch': 3.0}


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at Czunzun/finBERT2_v2 and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.627300
1000,0.544900
1500,0.351400
2000,0.311700
2500,0.162100


{'eval_loss': 0.6951795220375061, 'eval_accuracy': 0.8597938144329897, 'eval_cross_entropy': 0.6966099646543678, 'eval_f1': 0.8424264741159958, 'eval_runtime': 3.8924, 'eval_samples_per_second': 249.716, 'eval_steps_per_second': 62.429, 'epoch': 3.0}
